# Query data based on multiple conditions

## Package imports

In [ ]:
import sys

sys.path.append("..")
from pytreedb import db

Define (local) MongoDB connection and import database from URL

In [ ]:
mydbfile = "my_complex_query_pytree.db"
db_url = "https://github.com/3dgeo-heidelberg/pytreedb/raw/main/data/test/data.db"
mydb = db.PyTreeDB(dbfile=mydbfile)
mydb.import_db(db_url, overwrite=False)

## Queries with logical operator between conditions